In [19]:
import data_processing.port_n_vessels as port
import data_processing.weather as weather
import duckdb
from datetime import datetime, timedelta
import pandas as pd
from datetime import datetime

In [20]:
crawler_dict = port.crawler()
hourly = weather.openmeteo_hourly()

c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https

In [21]:
scheduled = crawler_dict['scheduled']['df']
scheduled['Date'] = pd.to_datetime(scheduled['Date'], format='mixed',dayfirst=True)
scheduled['ETA'] = pd.to_datetime(scheduled['ETA'], format='mixed', dayfirst=True)
scheduled['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

foreseen_cargo = crawler_dict['foreseen_cargo']['df']
foreseen_cargo['Arrival'] = pd.to_datetime(foreseen_cargo['Arrival'], format='mixed', dayfirst=True)
foreseen_cargo['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

foreseen_cruise = crawler_dict['foreseen_cruise']['df']
foreseen_cruise['Arrival'] = pd.to_datetime(foreseen_cruise['Arrival'], format='mixed', dayfirst=True)
foreseen_cruise['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

berthage_ships = crawler_dict['berthage_ships']['df']
berthage_ships['Arrival'] = pd.to_datetime(berthage_ships['Arrival'], format='mixed', dayfirst=True)
berthage_ships['Ship'] = berthage_ships['Ship'].str.replace(r'PROGRAMADO', '', case=False, regex=True)
berthage_ships['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

berthed_ships = crawler_dict['berthed_ships']['df']
berthed_ships['Load'] = berthed_ships['Load'].astype(int)
berthed_ships['Unload'] = berthed_ships['Unload'].astype(int)
berthage_ships['UPDATED_AT'] = datetime.now()

In [22]:
con_duck = duckdb.connect(database='db.duckdb')

In [23]:
con_duck.sql("SELECT * FROM duckdb_tables;").df()

,database_name,database_oid,schema_name,schema_oid,table_name,table_oid,comment,tags,internal,temporary,has_primary_key,estimated_size,column_count,index_count,check_constraint_count,sql
0,db,1146,main,1148,berthage_ships,1151,None,{},False,False,False,0,15,0,0,"CREATE TABLE berthage_ships(Ship VARCHAR, Flag..."
1,db,1146,main,1148,foreseen_cargo,1157,None,{},False,False,False,0,15,0,0,"CREATE TABLE foreseen_cargo(Ship VARCHAR, Flag..."
2,db,1146,main,1148,foreseen_cruise,1163,None,{},False,False,False,0,13,0,0,"CREATE TABLE foreseen_cruise(Ship VARCHAR, Fla..."
3,db,1146,main,1148,scheduled,1169,None,{},False,False,False,0,12,0,0,"CREATE TABLE scheduled(Date TIMESTAMP, ""Hour"" ..."
4,db,1146,main,1148,weather_data,1175,None,{},False,False,False,0,15,0,0,"CREATE TABLE weather_data(date TIMESTAMP, temp..."


In [24]:
con_duck.close()